In [ ]:
import numpy as np
import scipy


import kwant

import holoviews as hv
hv.notebook_extension()

#### check kwant version and use proper vector factory

In [ ]:
from functools import partial
from codes.kpm_funcs import build_greens_function, build_perturbation, proj, exact_greens_function

### definition of initial Hamiltonian and perturbation

In [ ]:
def H0_random(nA=4, nB=100, gap=1, epsilonA=0.2, epsilonB=10):
    """Generate random Hamiltonian with quasi-degenerate states."""
    energiesA = epsilonA * np.random.random(nA) - epsilonA / 2
    
    energiesB = epsilonB * np.random.random(nB) - epsilonB / 2
    energiesB = energiesB[np.abs(energiesB) > gap/2]

    energies = np.append(energiesA, energiesB)
    U = kwant.rmt.circular(len(energies))
    
    return np.diag(energies)#U.transpose().conjugate() @ np.diag(energies) @ U


def H1_random(n, v=0.1):
    return kwant.rmt.gaussian(n, v=v)

alphas = np.linspace(0, 1, 100)
np.random.seed(1)

H0 = H0_random()        # This is H_0
H1 = H1_random(len(H0)) # This is perturbation (H')

energies = []
for alpha in alphas:
    e = np.linalg.eigh(H0 + alpha * H1)[0]
    energies.append(e)

In [ ]:
H0.shape

In [ ]:
window = (-.25, +.25)
eigenvalues0, evec = np.linalg.eigh(H0)

indices = [i for (i, e) in enumerate(eigenvalues0) if window[0] < e < window[1]]
n = len(indices)

In [ ]:
bounds = (np.min(eigenvalues0)-0.05, np.max(eigenvalues0) + 0.05)

In [ ]:
hv.Path((alphas, energies))[:,window[0]:window[1]]

### kpm greens function

### compare with exact calculation of Green's funciton

In [ ]:
init_vector = np.exp(2j*np.pi*np.random.rand(2, H0.shape[0]))
# init_vector = init_vector / np.linalg.norm(init_vector)

In [ ]:
energies = np.linspace(bounds[0],bounds[1],200)

In [ ]:
e = 0

g_exact = exact_greens_function(H0)
expanded_init_vector_exact = g_exact(init_vector, [e]*init_vector.shape[0])

In [ ]:
init_vector.shape, expanded_init_vector_exact.shape

In [ ]:
g_kpm = build_greens_function(H0, vectors=init_vector,
                              kpm_params=dict(
                                  bounds=bounds,
                                  num_moments=600)
                             )
expanded_init_vectors_kpm = g_kpm(e)

In [ ]:
expanded_init_vectors_kpm.shape

In [ ]:
# average quotient of the two vectors
np.mean(expanded_init_vector_exact/expanded_init_vectors_kpm, axis=1)

#### compare absolute value and phase for the Green's function applied to one vector 

In [ ]:
expanded_init_vector_exact.shape, expanded_init_vectors_kpm.shape

In [ ]:
hv.Curve(np.linalg.norm(expanded_init_vector_exact, axis=0)) * hv.Curve(np.linalg.norm(expanded_init_vectors_kpm, axis=0))

In [ ]:
hv.Curve(np.angle(expanded_init_vector_exact[0])) * hv.Curve(np.angle(expanded_init_vectors_kpm[0])) +\
hv.Curve(np.angle(expanded_init_vector_exact[1])) * hv.Curve(np.angle(expanded_init_vectors_kpm[1]))

#### Green's function with exact and kpm methods

In [ ]:
%%opts Scatter (color='r')
g_exact = exact_greens_function(H0)
g_sum_exact = []
for e in energies:
    expanded_vector_exact = g_exact(init_vector, [e]*init_vector.shape[0])
    g_sum_exact.append(np.sum(init_vector.conj() * expanded_vector_exact, axis=1))
g_sum_exact = np.array(g_sum_exact)
hv.Path((energies, g_sum_exact.real)) * hv.Scatter((np.linalg.eigh(H0)[0], [0]*H0.shape[0]))

In [ ]:
%%opts Scatter (color='r')
# density of states = - 1/pi Im(G)
hv.Path((energies,-1/np.pi*g_sum_exact.imag)) * hv.Scatter((np.linalg.eigh(H0)[0], [0]*H0.shape[0]))

In [ ]:
%%opts Scatter (color='r')

g_kpm = build_greens_function(
    H0, vectors=init_vector,
    kpm_params=dict(bounds=bounds, num_moments=1000))

g_sum_kpm = []
for e in energies:
    expanded_vectors_kpm = g_kpm(e)
    g_sum_kpm.append(np.sum(init_vector.conj() * expanded_vectors_kpm, axis=1) )
g_sum_kpm = np.array(g_sum_kpm)
hv.Path((energies, g_sum_kpm.real))[:,:] * hv.Scatter((np.linalg.eigh(H0)[0], [0]*H0.shape[0]))

In [ ]:
%%opts Scatter (color='r')

g_kpm = build_greens_function(
    H0, vectors=init_vector,
    kpm_params=dict(bounds=bounds, num_moments=1000))

g_sum_kpm = []
expanded_vectors_kpm = g_kpm(energies)
g_sum_kpm = np.sum(init_vector.conj()[None,] * expanded_vectors_kpm, axis=2)
hv.Path((energies, g_sum_kpm.real))[:,:] * hv.Scatter((np.linalg.eigh(H0)[0], [0]*H0.shape[0]))

In [ ]:
expanded_vectors_kpm.shape, g_sum_kpm.shape

In [ ]:
%%opts Scatter (color='r')
# density of states = -1\pi Im(G)
hv.Path((energies, -1/np.pi*g_sum_kpm.imag)) * hv.Scatter((np.linalg.eigh(H0)[0], [0]*H0.shape[0]))

#### build projectors over a subspace

In [ ]:
window = (-.25, +.25)
eigenvalues0, evec = np.linalg.eigh(H0)

indices = [i for (i, e) in enumerate(eigenvalues0) if window[0] < e < window[1]]
n = len(indices)

In [ ]:
eigs_subspace = eigenvalues0[indices]
vecs_subspace = evec[:, indices].T
init_vector = (H1 @ vecs_subspace.T).T #np.exp(2j*np.pi*np.random.rand(1, H0.shape[0]))

In [ ]:
init_vector.shape, vecs_subspace.shape

In [ ]:
proj(init_vector, vecs_subspace).shape

In [ ]:
init_vector.shape

In [ ]:
hv.Scatter(np.abs(init_vector).flatten())

In [ ]:
hv.Scatter(np.abs(proj(init_vector, vecs_subspace)).flatten())

In [ ]:
np.linalg.norm(proj(init_vector, vecs_subspace))

### perturbation elements of the matrix

In [ ]:
eigenvalues0, psi0 = np.linalg.eigh(H0)
kpm_params = dict(num_moments=1000, bounds=(-5,5))

In [ ]:
eigenvalues0[indices]

In [ ]:
psi0[:,indices].shape

In [ ]:
hij2 = build_perturbation(eigenvalues0[indices], psi0[:,indices], H0, H1, kpm_params=kpm_params)

In [ ]:
hij2.shape

In [ ]:
np.allclose(hij2, hij2.conj().T)

In [ ]:
hij2

#### apply to a specific realization of $H_0$ and $H$

In [ ]:
eigenvalues0, psi0 = np.linalg.eigh(H0)

In [ ]:
rescaled_ham, (a0, b0) = kwant.kpm._rescale(H0, 0.0005, None, None)
rescaled_ham, (a1, b1) = kwant.kpm._rescale(H0 + H1, 0.0005, None, None)

In [ ]:
a0, a1, b0, b1

In [ ]:
kpm_params = dict(bounds=(b0-a0, a0+b0))

In [ ]:
eigenvalues, psi = np.linalg.eigh(H0 + H1)

In [ ]:
window = (-.25, +.25)
ev, evec = np.linalg.eigh(H0)

indices = [i for (i, e) in enumerate(ev) if window[0] < e < window[1]]
n = len(indices)

#### first order correction

In [ ]:
hij1 = psi0[:,indices].conj().T @ (H1 @ psi0[:,indices])

In [ ]:
hij1.shape

In [ ]:
np.allclose(hij1, hij1.conj().T)

#### second order correction

In [ ]:
kpm_params

In [ ]:
kpm_params['num_moments'] = 1000

In [ ]:
hij2 = build_perturbation(eigenvalues0[indices], psi0[:,indices], H0, H1, kpm_params=kpm_params)

In [ ]:
hij2.shape

In [ ]:
np.allclose(hij2, hij2.conj().T)

In [ ]:
hij2

In [ ]:
h_eff = np.diag(eigenvalues0[indices]) + hij1 + hij2

In [ ]:
e_eff, psi_eff = np.linalg.eigh(h_eff)

In [ ]:
# h1_eff and h2_eff

eigs_exact = []
eigs_eff1 = []
eigs_eff2 = []
alphas = np.logspace(-8, 0, 200)
for alpha in alphas:
    eigs_exact.append(np.linalg.eigh(H0 + alpha * H1)[0][indices])
    eigs_eff1.append(np.linalg.eigh(np.diag(ev[indices])+
                                   alpha * hij1
                                  )[0]
                   )
    eigs_eff2.append(np.linalg.eigh(np.diag(ev[indices])+
                                   alpha * hij1 +
                                   alpha**2 * hij2
                                  )[0]
                   )
eigs_exact = np.array(eigs_exact)
eigs_eff1 = np.array(eigs_eff1)
eigs_eff2 = np.array(eigs_eff2)

In [ ]:
(
    hv.Path((alphas, eigs_exact), kdims=[r'$\alpha$', r'$\epsilon$']) *
    hv.Path((alphas, eigs_eff1), kdims=[r'$\alpha$', r'$\epsilon$']) *
    hv.Path((alphas, eigs_eff2), kdims=[r'$\alpha$', r'$\epsilon$'])
)[:,-0.35:0.35]

In [ ]:
%%opts Overlay [logy=False]
(
    hv.Path((alphas, np.abs(eigs_exact-eigs_eff1)),
           kdims=[r'$\alpha$', 'diff']) *
    hv.Path((alphas, np.abs(eigs_exact-eigs_eff2)),
           kdims=[r'$\alpha$', 'diff'])
)[:,1e-8:]

In [ ]:
%%opts Overlay [logx=True logy=True]
(
    hv.Path((alphas, np.abs(eigs_exact-eigs_eff1)),
           kdims=[r'$\alpha$', 'diff']) *
    hv.Path((alphas, np.abs(eigs_exact-eigs_eff2)),
            kdims=[r'$\alpha$', 'diff'])
)[1e-8:,1e-16:]